In [1]:
import os

In [2]:
%pwd

'f:\\end-to-end-deep_learning-project\\research'

In [3]:
os.chdir("../")
%pwd

'f:\\end-to-end-deep_learning-project'

# update config.yaml

# entity

In [23]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [24]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [ ]:
import os
#import urllib.request as request
#from zipfile import ZipFile
import torch
import torch.nn as nn
import torchvision.models as models


In [30]:
class PrepareBaseModel:

    def __init__(self, config:PrepareBaseModelConfig):
        self.config = config
        


    def get_base_model(self):
        """Load VGG16 + freeze convolution layers + save base model"""
        

        # load pretrained VGG16
        # Convert YAML string → PyTorch Weight Enum
        weights_enum = getattr(models.VGG16_Weights, self.config.params_weights)
    
        model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

        # freeze feature extractor layers
        for param in model.features.parameters():
            param.requires_grad = False

        # save base model (frozen)
        self.save_model(model,self.config.base_model_path)
        

    def update_base_model(self):
        """Load base model + attach custom classifier + save updated model"""

        # load base model from file (important!)
        model = torch.load(self.config.base_model_path)

        # replace classifier head
        model.classifier = nn.Sequential(
            nn.Linear(25088, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, self.config.params_classes)
        )

        # ensure classifier layers are trainable
        for param in model.classifier.parameters():
            param.requires_grad = True

        # save updated model (ready for training)
        self.save_model(model,self.config.updated_base_model_path)
        print(model)
        


    def save_model(self, model, path):
        """Save full PyTorch model"""
        torch.save(model, path)


In [31]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-12-10 13:33:55,125: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-10 13:33:55,133: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-10 13:33:55,135: INFO: common: created directory at: artifacts]
[2025-12-10 13:33:55,138: INFO: common: created directory at: artifacts/prepare_base_model]
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), 